In [ ]:
import pandas as pd
import numpy as np
import re

def convert_csv_kgraw_format(file_path, gene_column, fc_column, edge_label, paper, cellname, pval_column=None, pval_threshold=0.05):
    """
    Load a dataset, extract relevant columns, clean the data, and prepare it for ALzKG.

    Parameters:
    - file_path: str, the path to the dataset file.
    - gene_column: str, the column name containing gene names.
    - fc_column: str, the column name containing fold change values.
    - edge_label: str, the custom label to use as a prefix to up/ down
    - paper: str, paper name
    - cellname: str, celltype/state_name
    - pval_column: str (optional), the column name containing p-values.
    - pval_threshold: float (optional), threshold for filtering based on p-values.

    Returns:
    - A pandas DataFrame processed according to the specified parameters.
    """
    # Load the dataset
    data = pd.read_csv(file_path)

    # Extract relevant columns
    relevant_columns = [gene_column, fc_column] + ([pval_column] if pval_column is not None else [])
    data = data.loc[:, relevant_columns]

    # Clean gene names
    data = data[~data[gene_column].apply(clean_data)]

    # Filter rows based on p-value threshold and non-null criteria if pval_column is provided
    if pval_column is not None:
        data = data[(data[pval_column] < pval_threshold) & (~np.isnan(data[pval_column]))]
        data.drop(columns=[pval_column], inplace=True)  # Drop the pval_column after filtering

    # Convert fold change values to numeric and categorize with the custom edge label
    data[fc_column] = pd.to_numeric(data[fc_column], errors='coerce')
    data[fc_column] = data[fc_column].apply(lambda x: categorize_fold_change(x, edge_label))

    # Print unique edge types in CSV format with headers
    unique_edges = data[fc_column].unique()
    print('"edge_type","desc"')
    for edge in unique_edges:
       print(f'"{edge}","..."')

    # Rename columns
    data.rename(columns={gene_column: 'y_name', fc_column: 'relation'}, inplace=True)

    # Add new columns
    data['display_relation'] = data['relation'].apply(lambda x: shorten_relation(x))  # Define the 'shorten_relation' function based on your criteria
    data['y_id'] = 'tbd'
    data['y_type'] = 'gene/protein'
    data['y_source'] = 'NCBI'
    data['x_id'] = 'tbd'
    data['x_type'] = 'celltype/state'
    data['x_name'] = cellname
    data['x_source'] = paper

    # Reorder columns
    final_columns_order = ['relation', 'display_relation', 'x_id', 'x_type', 'x_name', 'x_source', 'y_id', 'y_type', 'y_name', 'y_source']
    data = data[final_columns_order]

    # Save the processed DataFrame to a new CSV file
    output_path = file_path.replace('.csv', '_kg.csv')
    data.to_csv(output_path, index=False)

def clean_data(name):
    # Function to check if the input is NaN or matches patterns that are not valid gene names.
    # Mathys data has a few rows with timestamps instead of gene names
    if pd.isna(name):
        return True
    elif isinstance(name, str) and re.match(r"^\d{1,2}-[A-Za-z]{3,4}$", name):
        return True
    return False

def categorize_fold_change(x, edge_label):
    """
    Categorize fold change values based on a custom edge label.

    Parameters:
    - x: float, the fold change value to categorize.
    - edge_label: str, the custom edge label to prepend to 'up' or 'down'.

    Returns:
    - str, a string indicating the categorization of the fold change.
    """
    if pd.isna(x):
        return 'nan'  # Adjust as needed for NaN values
    elif x > 0:
        return f'{edge_label} up'
    else:  # Assumes x < 0 since NaN is already handled
        return f'{edge_label} down'

def shorten_relation(relation):
    """
    Generates a generic abbreviation for the relation description by taking the
    first letter of each significant word in the phrase.

    Parameters:
    - relation: str, the full relation description to be abbreviated.

    Returns:
    - str, an abbreviated version of the relation description.
    """
    # List of words to exclude from the abbreviation (feel free to expand this list)
    exclude_words = {'and', 'of', 'to', 'in', 'for', 'with', 'the'}

    # Split the relation into words, exclude certain words, and then take the first letter of each remaining word
    abbreviation = ''.join(word[0].upper() for word in relation.split() if word.lower() not in exclude_words)

    return abbreviation



In [ ]:
#Example Usage
# Import the drive module from Google Colab for file access, mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# File path for the dataset to be added to PrimeKG, each celltype is being processed as separate CSV files
file_path = "/content/drive/My Drive/primekg_files/Mic.csv"

# Specify the columns for gene names, p-values, and fold change (can be one or more columns dependent on dataset)
gene_column = 'Gene'
pval_column = 'IndModel.adj.pvals'
fc_column = 'IndModel.FC'

# Process the dataset
convert_csv_kgraw_format(
    file_path=file_path,
    gene_column=gene_column,
    fc_column=fc_column,
    edge_label = "no-pathology vs pathology mathys",
    paper = "Mathys et al 2019",
    cellname = "Mic",
    pval_column=pval_column,
    pval_threshold=0.05
)
